# Homework 5 : WeatherPy using Openweathermap API and Citipy

In [142]:
import openweathermapy.core as ow
from citipy import citipy as cp
import pandas as pd
import numpy as np
import random
import requests as req
import json
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [143]:
# Save config information.
api_key = "bfe8624363d9fae0c73d21055be1bfe8" #my weather api
#api_key = "25bc90a1196e6f153eece0bc0b0fc9eb" 
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

# Build partial query URL
query_url = url + "appid=" + api_key + "&units=" + units + "&q="            

In [156]:
# Create dictionary of random coordinates; testing shows that range of 1500 will give <800 unique cities from Citipy
lat = []
lon = []
random_coords = {'Latitude': lat,'Longitude': lon}
for i in range(0,1200): 
    ran_lat = random.uniform(-90, 90)
    lat.append(ran_lat)  
    ran_lon = random.uniform(-180,180)
    lon.append(ran_lon)

In [157]:
# Check number of coordinates
len(random_coords['Latitude'])

1200

In [158]:
#Create dataframe for coordinates
df = pd.DataFrame(random_coords)
df.head(3)

,Latitude,Longitude
0,-52.489568,-136.025004
1,-58.271318,20.308320
2,56.255506,90.052235


In [159]:
# Create additional colums for City, Temperature (F), Humidity (%), Cloudiness (%), Wind Speed (mph)
df['City'] = ""
df['Country'] = ""
df['Temperature (F)'] = ""
df['Humidity (%)'] = ""
df['Cloudiness (%)'] = ""
df['Wind Speed (mph)'] = ""
df['Lat'] = ""
df['Lon'] = ""
df.head(2)

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
0,-52.489568,-136.025004,,,,,,,,
1,-58.271318,20.308320,,,,,,,,


In [160]:
# Loop through coordinates in df with Citipy to return cities
row_count = 1
for index, row in df.iterrows():
   
    city = cp.nearest_city((row["Latitude"]), (row["Longitude"]))
    return_city = city.city_name
    return_country = city.country_code
    
    df.set_value(index, "City", return_city)
    df.set_value(index, "Country", return_country) 
     
    # Print log to ensure loop is working correctly
    #print("Now retrieving city # " + str(row_count) + " " + return_city + ", " + return_country)
    row_count += 1 
    
df.head()

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
0,-52.489568,-136.025004,rikitea,pf,,,,,,
1,-58.271318,20.308320,bredasdorp,za,,,,,,
2,56.255506,90.052235,mazulskiy,ru,,,,,,
3,-53.836110,-59.127966,ushuaia,ar,,,,,,
4,82.030927,170.069533,pevek,ru,,,,,,


In [161]:
# Check number of unique (non-repeating) cities were generated
len(df["City"].unique())

504

In [162]:
# Visual check and length check duplicated cities
df.sort_values("City", inplace=True)
mask=df["City"].duplicated()
df[mask].head()
#len(df[mask])

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
32,4.566399,-105.035302,acapulco,mx,,,,,,
792,9.656470,138.805303,airai,pw,,,,,,
15,14.766983,138.001900,airai,pw,,,,,,
281,78.422735,-141.937114,aklavik,ca,,,,,,
478,71.832733,-141.563255,aklavik,ca,,,,,,


In [163]:
# There are alot of duplicates since Citipy tries to match to random coords to nearest city
# Drop duplicates and check that results count is same as unique
df.drop_duplicates(subset = ["City"], keep = 'first', inplace=True) # 'first keeps first occurence 
len(df)

504

In [164]:
# Loop through all cities using OpenWeatherMap to return weather info
# Need to overwrite random-generated coords with actual coords since plotting random-generated lat not same as city lat
row_count = 1
for index, row in df.iterrows():
    
    city = row["City"]
    target_url = (query_url + city)
    
    # Print log to ensure loop is working correctly
    print("Now retrieving city # " + str(row_count) + " " + city.title() + ": " + target_url)
    row_count += 1 
    
    weather_data = [req.get(target_url).json()]
    return_temp = weather_data[0]["main"]["temp"]
    return_humid = weather_data[0]["main"]["humidity"]
    return_cloud = weather_data[0]["clouds"]["all"]
    return_wind = weather_data[0]["wind"]["speed"]
    return_lat = weather_data[0]["coord"]["lat"]
    return_lon = weather_data[0]["coord"]["lon"]
    
    df.set_value(index, "Temperature (F)", return_temp)
    df.set_value(index, "Humidity (%)", return_humid) 
    df.set_value(index, "Cloudiness (%)", return_cloud)
    df.set_value(index, "Wind Speed (mph)", return_wind)         
    df.set_value(index, "Lat", return_lat)
    df.set_value(index, "Lon", return_lon)

Now retrieving city # 1 Abeche: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=abeche
Now retrieving city # 2 Abu Samrah: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=abu samrah
Now retrieving city # 3 Acajutla: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=acajutla
Now retrieving city # 4 Acapulco: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=acapulco
Now retrieving city # 5 Acari: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=acari
Now retrieving city # 6 Acu: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=acu
Now retrieving city # 7 Adilabad: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=adi

Now retrieving city # 58 Barrow: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=barrow
Now retrieving city # 59 Basco: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=basco
Now retrieving city # 60 Batagay: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=batagay
Now retrieving city # 61 Bathsheba: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=bathsheba
Now retrieving city # 62 Bayir: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=bayir
Now retrieving city # 63 Baykit: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=baykit
Now retrieving city # 64 Beidao: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=be

Now retrieving city # 114 Codrington: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=codrington
Now retrieving city # 115 Conceicao Do Mato Dentro: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=conceicao do mato dentro
Now retrieving city # 116 Constitucion: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=constitucion
Now retrieving city # 117 Cooma: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=cooma
Now retrieving city # 118 Coquimbo: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=coquimbo
Now retrieving city # 119 Coron: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=coron
Now retrieving city # 120 Corvallis: http://api.openweathermap.org/data/2.5/weat

Now retrieving city # 170 Greenfield: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=greenfield
Now retrieving city # 171 Grindavik: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=grindavik
Now retrieving city # 172 Guaranda: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=guaranda
Now retrieving city # 173 Guerrero Negro: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=guerrero negro
Now retrieving city # 174 Gulshat: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=gulshat
Now retrieving city # 175 Gumdag: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=gumdag
Now retrieving city # 176 Haapiti: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9

Now retrieving city # 226 Kato Khorion: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=kato khorion
Now retrieving city # 227 Katsuura: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=katsuura
Now retrieving city # 228 Kaunas: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=kaunas
Now retrieving city # 229 Kavieng: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=kavieng
Now retrieving city # 230 Kazerun: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=kazerun
Now retrieving city # 231 Kemijarvi: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=kemijarvi
Now retrieving city # 232 Keshan: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d210

Now retrieving city # 282 Mao: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=mao
Now retrieving city # 283 Mar Del Plata: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=mar del plata
Now retrieving city # 284 Maragogi: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=maragogi
Now retrieving city # 285 Marataizes: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=marataizes
Now retrieving city # 286 Marcona: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=marcona
Now retrieving city # 287 Maroantsetra: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=maroantsetra
Now retrieving city # 288 Marrakesh: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9

Now retrieving city # 338 Nikolskoye: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=nikolskoye
Now retrieving city # 339 Nizhneyansk: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=nizhneyansk
Now retrieving city # 340 Nizza Monferrato: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=nizza monferrato
Now retrieving city # 341 Nome: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=nome
Now retrieving city # 342 Norman Wells: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=norman wells
Now retrieving city # 343 Nouakchott: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=nouakchott
Now retrieving city # 344 Ntoum: http://api.openweathermap.org/data/2.5/weather?ap

Now retrieving city # 394 Roanoke: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=roanoke
Now retrieving city # 395 Rockland: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=rockland
Now retrieving city # 396 Rybnoye: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=rybnoye
Now retrieving city # 397 Safaqis: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=safaqis
Now retrieving city # 398 Sahrak: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=sahrak
Now retrieving city # 399 Saint George: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=saint george
Now retrieving city # 400 Saint-Augustin: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73

Now retrieving city # 449 Tiksi: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=tiksi
Now retrieving city # 450 Tolaga Bay: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=tolaga bay
Now retrieving city # 451 Tomohon: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=tomohon
Now retrieving city # 452 Tondano: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=tondano
Now retrieving city # 453 Toora-Khem: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=toora-khem
Now retrieving city # 454 Torbay: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be1bfe8&units=imperial&q=torbay
Now retrieving city # 455 Torrevieja: http://api.openweathermap.org/data/2.5/weather?appid=bfe8624363d9fae0c73d21055be

KeyError: 'main'

In [165]:
df.head()

,Latitude,Longitude,City,Country,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lon
671,13.235295,20.842553,abeche,td,69.74,100,64,4.38,13.83,20.83
989,22.724199,50.521941,abu samrah,qa,74.33,82,80,7.74,35.17,37.25
743,13.087621,-89.994301,acajutla,sv,74.78,95,48,2.37,13.59,-89.83
588,14.973282,-100.329832,acapulco,mx,82.4,78,90,6.93,16.86,-99.89
1110,-16.296284,-75.231415,acari,pe,46.07,77,0,2.15,-15.43,-74.62


In [167]:
len(df)

504